# Riskfolio-Lib Tutorial: 
<br>__[Financionerioncios](https://financioneroncios.wordpress.com)__
<br>__[Orenji](https://www.orenj-i.net)__
<br>__[Riskfolio-Lib](https://riskfolio-lib.readthedocs.io/en/latest/)__
<br>__[Dany Cajas](https://www.linkedin.com/in/dany-cajas/)__
<a href='https://ko-fi.com/B0B833SXD' target='_blank'><img height='36' style='border:0px;height:36px;' src='https://cdn.ko-fi.com/cdn/kofi1.png?v=2' border='0' alt='Buy Me a Coffee at ko-fi.com' /></a> 

## Tutorial 17: Riskfolio-Lib with MOSEK for Real Applications (612 assets and 4943 observations)

This tutorial shows how to use Riskfolio-Lib in real applications, when problems are large scale. This is a situation that banks, hedge funds, mutual funds and others institutional investors face daily.

## 1. Downloading the data:

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.4%}'.format

data = pd.read_csv("assets_data.csv", index_col='Dates')

Y = data.pct_change().dropna()

display(Y.head())

,A,AAPL,ABC,ABMD,ABT,ADBE,ADM,ADSK,AEE,AEP,...,WWD,X,Y,ZBRA,ASML,BMRN,CHKP,MXIM,SHPG,SIRI
Dates,,,,,,,,,,,,,,,,,,,,,
04-01-2000,-8.3842%,-8.4321%,-6.8261%,-2.3945%,-1.9714%,-8.3890%,-1.0419%,-4.8695%,0.0000%,1.1928%,...,0.4755%,-3.7328%,-0.5362%,-1.4428%,-6.6594%,6.3725%,-9.0390%,-3.6943%,-5.6250%,-6.6869%
05-01-2000,-4.7666%,1.4642%,7.7584%,1.0498%,-0.7307%,1.9773%,-1.5792%,-6.2992%,3.8685%,3.7328%,...,-2.8302%,1.0204%,-0.3703%,1.9143%,-2.7618%,-5.0691%,-1.0062%,0.2646%,5.5188%,5.0489%
06-01-2000,-5.7057%,-8.6530%,7.6008%,0.1722%,1.6572%,0.8163%,0.5349%,-6.0921%,-0.3724%,0.5682%,...,-1.2140%,0.6061%,0.5404%,-5.4143%,-6.1538%,-0.9709%,-6.8694%,2.7704%,5.6485%,2.9457%
07-01-2000,10.4035%,4.7364%,13.0097%,-0.5158%,3.9857%,4.8583%,1.5960%,11.8564%,1.4953%,1.3183%,...,0.2453%,2.6104%,2.7229%,-1.1099%,2.2697%,4.9020%,4.7156%,2.3107%,8.9109%,-1.5060%
10-01-2000,6.0575%,-1.7588%,2.6318%,14.2825%,-2.2653%,3.8610%,0.0000%,-1.1994%,-0.7366%,-0.3717%,...,-6.1271%,-0.7828%,-1.5378%,3.3669%,9.4945%,2.3364%,10.3583%,1.3802%,-0.7273%,3.9755%


## 2. Estimating Max Mean/Risk Portfolios for all Risk Measures

Instructions to install MOSEK are in this __[link](https://docs.mosek.com/9.2/install/installation.html)__, is better to install using Anaconda. Also you will need a license, I recommend you that ask for an academic license __[here](https://www.mosek.com/products/academic-licenses/)__.

In [2]:
import riskfolio as rp

# Building the portfolio object
port = rp.Portfolio(returns=Y)

# Calculating optimum portfolio

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

# Input model parameters:

port.solvers = ['MOSEK'] # Setting MOSEK as the default solver
# if you want to set some MOSEK params use this code as an example
# import mosek
# port.sol_params = {'MOSEK': {'mosek_params': {mosek.iparam.intpnt_solve_form: mosek.solveform.dual}}}

port.alpha = 0.05 # Significance level for CVaR, EVaR y CDaR 
model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

### 2.1 Optimizing Process by Risk Measure

In [3]:
from timeit import default_timer as timer
from datetime import timedelta

# Risk Measures available:
#
# 'MV': Standard Deviation.
# 'MAD': Mean Absolute Deviation.
# 'MSV': Semi Standard Deviation.
# 'FLPM': First Lower Partial Moment (Omega Ratio).
# 'SLPM': Second Lower Partial Moment (Sortino Ratio).
# 'CVaR': Conditional Value at Risk.
# 'EVaR': Entropic Value at Risk.
# 'WR': Worst Realization (Minimax)
# 'MDD': Maximum Drawdown of uncompounded cumulative returns (Calmar Ratio).
# 'ADD': Average Drawdown of uncompounded cumulative returns.
# 'CDaR': Conditional Drawdown at Risk of uncompounded cumulative returns.
# 'EDaR': Entropic Drawdown at Risk of uncompounded cumulative returns.
# 'UCI': Ulcer Index of uncompounded cumulative returns.

rms = ["MV", "MAD", "MSV", "FLPM", "SLPM", "CVaR",
       "EVaR", "WR", "MDD", "ADD", "CDaR", "UCI", "EDaR"]

w = {}
for rm in rms:
    start = timer()
    w[rm] = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    end = timer()
    print(rm + ' takes ',timedelta(seconds=end-start))

MV takes  0:00:01.901754
MAD takes  0:00:25.496272
MSV takes  0:00:41.408623
FLPM takes  0:00:16.244342
SLPM takes  0:01:02.089203
CVaR takes  0:00:22.155717
EVaR takes  0:02:49.225111
WR takes  0:00:15.233740
MDD takes  0:00:57.069299
ADD takes  0:00:40.580395
CDaR takes  0:01:01.629065
UCI takes  0:00:44.314400
EDaR takes  0:01:23.120478


### 2.2 Portfolio Weights

In [4]:
w_s = pd.DataFrame([])
for rm in rms:
    w_s = pd.concat([w_s, w[rm]], axis=1)

w_s.columns = rms

display(w_s)

,MV,MAD,MSV,FLPM,SLPM,CVaR,EVaR,WR,MDD,ADD,CDaR,UCI,EDaR
A,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
AAPL,2.7531%,2.3444%,2.8936%,2.0108%,2.9583%,3.7963%,4.9669%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABC,2.3723%,0.7075%,3.6587%,1.1215%,3.8778%,2.9236%,10.5758%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABMD,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.3634%,0.0000%
ABT,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
BMRN,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
CHKP,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
MXIM,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
SHPG,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%


### 2.3 In sample CAGR by Portfolio

In [5]:
from datetime import datetime
a1 = datetime.strptime(data.index[0], '%d-%m-%Y')
a2 = datetime.strptime(data.index[-1], '%d-%m-%Y')
days = (a2-a1).days

cagr = {} 
for rm in rms:
    a = np.prod(1 + Y @ w_s[rm]) ** (360/days)-1
    cagr[rm] = [a]

cagr = pd.DataFrame(cagr).T
cagr.columns = ['CAGR']

cagr.style.format("{:.2%}").background_gradient(cmap='RdYlGn')

,CAGR
MV,28.20%
MAD,28.01%
MSV,28.65%
FLPM,28.07%
SLPM,28.68%
CVaR,27.91%
EVaR,29.53%
WR,26.71%
MDD,28.82%
ADD,25.91%


## 3. Estimating Min Risk Portfolios for all Risk Measures

### 3.1 Optimizing Process by Risk Measure

In [6]:
rms = ["MV", "MAD", "MSV", "FLPM", "SLPM", "CVaR",
       "EVaR", "WR", "MDD", "ADD", "CDaR", "UCI", "EDaR"]

w_min = {}
obj = 'MinRisk'
for rm in rms:
    start = timer()
    w_min[rm] = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    end = timer()
    print(rm + ' takes ',timedelta(seconds=end-start))

MV takes  0:00:01.150457
MAD takes  0:00:13.393403
MSV takes  0:00:19.970606
FLPM takes  0:00:12.554464
SLPM takes  0:00:35.985195
CVaR takes  0:00:14.679440
EVaR takes  0:01:26.016303
WR takes  0:00:11.170362
MDD takes  0:00:40.826052
ADD takes  0:00:27.828867
CDaR takes  0:00:58.208066
UCI takes  0:00:41.163487
EDaR takes  0:01:16.967535


### 3.2 Portfolio Weights

In [7]:
w_min_s = pd.DataFrame([])
for rm in rms:
    w_min_s = pd.concat([w_min_s, w_min[rm]], axis=1)

w_min_s.columns = rms

display(w_min_s)

,MV,MAD,MSV,FLPM,SLPM,CVaR,EVaR,WR,MDD,ADD,CDaR,UCI,EDaR
A,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
AAPL,0.0000%,0.1746%,0.0000%,0.5334%,0.0000%,0.0000%,2.3313%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABC,0.0000%,0.0000%,0.0000%,0.1824%,0.0746%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABMD,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABT,0.0002%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
BMRN,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
CHKP,0.0001%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,3.1073%,4.2516%,3.6153%,0.0000%
MXIM,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,1.9335%,4.4013%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
SHPG,0.0001%,0.8500%,0.0000%,0.4403%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%


### 3.3 In sample CAGR by Portfolio

In [8]:
from datetime import datetime
a1 = datetime.strptime(data.index[0], '%d-%m-%Y')
a2 = datetime.strptime(data.index[-1], '%d-%m-%Y')
days = (a2-a1).days

min_cagr = {} 
for rm in rms:
    a = np.prod(1 + Y @ w_min_s[rm]) ** (360/days)-1
    min_cagr[rm] = [a]

min_cagr = pd.DataFrame(min_cagr).T
min_cagr.columns = ['CAGR']

min_cagr.style.format("{:.2%}").background_gradient(cmap='RdYlGn')

,CAGR
MV,10.79%
MAD,9.87%
MSV,10.43%
FLPM,12.26%
SLPM,11.69%
CVaR,12.30%
EVaR,10.52%
WR,9.98%
MDD,16.86%
ADD,19.01%


### 2.3 Calculate efficient frontier

## 4. Estimating Risk Parity Portfolios for all Risk Measures

### 4.1 Optimizing Process by Risk Measure

In [9]:
rms = ["MV", "MAD", "MSV", "FLPM", "SLPM",
       "CVaR", "EVaR", "CDaR", "UCI", "EDaR"]

b = None # Risk contribution constraints vector, when None is equally risk per asset

w_rp = {}
for rm in rms:
    start = timer()
    

    w_rp[rm] = port.rp_optimization(model=model, rm=rm, rf=rf, b=b, hist=hist)
    end = timer()
    print(rm + ' takes ',timedelta(seconds=end-start))

MV takes  0:00:01.860670
MAD takes  0:00:12.328205
MSV takes  0:00:19.464142
FLPM takes  0:00:17.847353
SLPM takes  0:00:41.426538
CVaR takes  0:00:27.225739
EVaR takes  0:01:44.481045
CDaR takes  0:01:00.881278
UCI takes  0:00:44.719583
EDaR takes  0:01:23.270679


### 4.2 Portfolio Weights

In [10]:
w_rp_s = pd.DataFrame([])
for rm in rms:
    w_rp_s = pd.concat([w_rp_s, w_rp[rm]], axis=1)

w_rp_s.columns = rms

display(w_rp_s)

,MV,MAD,MSV,FLPM,SLPM,CVaR,EVaR,CDaR,UCI,EDaR
A,0.1246%,0.1182%,0.1220%,0.1132%,0.1205%,0.1279%,0.1482%,0.0593%,0.0625%,0.0729%
AAPL,0.1586%,0.1494%,0.1630%,0.1587%,0.1682%,0.1826%,0.2485%,0.0917%,0.1111%,0.1109%
ABC,0.2385%,0.2348%,0.2475%,0.2587%,0.2565%,0.2440%,0.2922%,0.1727%,0.1808%,0.1797%
ABMD,0.1326%,0.1215%,0.1272%,0.1280%,0.1301%,0.1373%,0.1564%,0.1085%,0.1298%,0.0901%
ABT,0.2834%,0.2660%,0.2686%,0.2643%,0.2701%,0.2652%,0.3321%,0.2586%,0.2635%,0.3459%
...,...,...,...,...,...,...,...,...,...,...
BMRN,0.1431%,0.1349%,0.1411%,0.1373%,0.1431%,0.1583%,0.1278%,0.1238%,0.1829%,0.0970%
CHKP,0.1515%,0.1365%,0.1643%,0.1359%,0.1658%,0.1890%,0.2059%,0.2106%,0.2244%,0.2744%
MXIM,0.1452%,0.1224%,0.1530%,0.1185%,0.1524%,0.1804%,0.2852%,0.0725%,0.0852%,0.0817%
SHPG,0.2129%,0.2091%,0.2068%,0.2092%,0.2089%,0.2193%,0.2489%,0.1130%,0.1254%,0.1295%


### 4.3 In sample CAGR

In [11]:
rp_cagr = {} 
for rm in rms:
    a = np.prod(1 + Y @ w_rp_s[rm]) ** (360/days)-1
    rp_cagr[rm] = [a]

rp_cagr = pd.DataFrame(rp_cagr).T
rp_cagr.columns = ['CAGR']

rp_cagr.style.format("{:.2%}").background_gradient(cmap='RdYlGn')

,CAGR
MV,12.93%
MAD,12.73%
MSV,12.92%
FLPM,12.95%
SLPM,13.02%
CVaR,13.13%
EVaR,13.10%
CDaR,15.93%
UCI,15.53%
EDaR,15.26%
